In [1]:
# TSP
import numpy as np

S = 0
N = 4
memo = np.zeros((N,2**N))
m =[[0,4,1,9],
    [3,0,6,11],
    [4,1,0,2],
    [6,5,-4,0]]


def not_in(i, subset):
    return (1 << i & subset) == 0


def find_opt_tour(m, memo, S, N):
    last = S
    
    # (N=4 => state = 1111)
    state = (1 << N) - 1

    # A Hamiltonian cycle
    tour = [-1 for _ in range(N+1)]
    
    # Set the beginning and start of the tour
    tour[0] = tour[N] = S
    
    # Fill in the intermediate nodes
    # Back to front
    for i in range(N-1,0,-1):
        node = -1
        for j in range(0, N):
            if j==S or not_in(j, state): continue
            node = j if node == -1 else node
            prev_dist = memo[node][state] + m[node][last]
            new_dist = memo[j][state] + m[j][last]
            node = j if new_dist < prev_dist else node
        tour[i] = node
        state = state ^ (1 << node)
        last = node
        
    return tour

def find_min_cost(m, memo, S, N):
    end_state = (1 << N) - 1
    min_cost = np.inf
    for e in range(N):
        if e == S: continue
        tour_cost = memo[e][end_state] + m[e][S]
        min_cost = min(tour_cost, min_cost)
    return min_cost

def combinations(r, n):
    if r > n:
        return []
    elif r == 0:
        return [0]
    elif r == n: 
        return [(1 << n) - 1]
    elif r < n:
        s0 = [ 0 | (option << 1) for option in combinations(r,n-1)]
        s1 = [ 1 | (option << 1) for option in combinations(r-1,n-1)] 
        return s0 + s1

    
def solve(m, memo, S, N):
    # Loop over all paths with length r
    for r in range(3,N+1):
        # Find all paths with r nodes out of N set
        for subset in combinations(r, N):
            # Continue if S not in Subset
            if not_in(S, subset): continue
            for new_end in range(0,N):
                if new_end == S or not_in(new_end, subset): continue
                # State will have r-1 nodes
                # These states we already calculated
                state = subset ^ (1 << new_end)
                min_dist = np.inf
                # e is end node
                for old_end in range(0,N):
                    if old_end == S or old_end == new_end or not_in(old_end,subset):continue
                    new_dist = memo[old_end][state] + m[old_end][new_end]
                    min_dist = min(min_dist, new_dist)
                memo[new_end][subset] = min_dist
                
def setup(m, memo, S, N):
    for i in range(N):
        if i == S: continue
        # Store the length of the path
        # from S to i
        memo[i][1 << S | 1 << i] = m[S][i]
        
def tsp(m, S):
    N = len(m)
    memo = [[0 for j in range(2**N)] for i in range(N)]
    setup(m, memo, S, N)
    solve(m, memo, S, N)
    min_cost = find_min_cost(m, memo, S, N)
    tour = find_opt_tour(m, memo, S, N)
    return min_cost, tour

print(tsp(m,0))

(9, [0, 3, 2, 1, 0])
